In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw05.ipynb")

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy
from pathlib import Path

plt.style.use('fivethirtyeight') # Use plt.style.available to see more styles
sns.set()
sns.set_context("talk")
%matplotlib inline

In [3]:

# Setup - Load the SQL extension and connect to the Mini IMDB dataset we've prepared
db_path = Path('data/imdbmini.db')

engine = sqlalchemy.create_engine(f"sqlite:///{db_path}")
connection = engine.connect()
inspector = sqlalchemy.inspect(engine)

#Get table names
inspector.get_table_names()


['Name', 'Rating', 'Role', 'Title']

In [4]:
query_name = """
SELECT * 
FROM Name
"""
pd.read_sql(query_name, engine).head()

,nconst,primaryName,birthYear,deathYear,primaryProfession
0,1,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous"
1,2,Lauren Bacall,1924,2014,"actress,soundtrack"
2,3,Brigitte Bardot,1934,None,"actress,soundtrack,music_department"
3,4,John Belushi,1949,1982,"actor,soundtrack,writer"
4,5,Ingmar Bergman,1918,2007,"writer,director,actor"


In [5]:
query_role = """
SELECT * 
FROM Rating
"""
pd.read_sql(query_role, engine).head()

,index,tconst,averageRating,numVotes
0,0,417,8.2,46382
1,1,4972,6.3,23836
2,2,10323,8.1,60244
3,3,12349,8.3,120059
4,4,13442,7.9,92218


In [6]:
query_rating = """
SELECT * 
FROM Role
"""
pd.read_sql(query_rating, engine).head()

,index,tconst,ordering,nconst,category,job,characters
0,0,417,1,617588,actor,None,"[""Prof. Barbenfouillis"",""The Moon""]"
1,1,417,2,29244,actor,None,"[""Astronomer""]"
2,2,417,3,76933,actress,None,"[""Lady in the Moon""]"
3,3,417,4,1215996,actor,None,"[""Astronomer""]"
4,4,417,5,894523,writer,"novel ""De la Terre à la Lune""",None


In [7]:
query_title = """
SELECT * 
FROM Title 
LIMIT 3
"""
pd.read_sql(query_title, engine).head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,417,short,A Trip to the Moon,Le voyage dans la lune,0,1902,None,13,"Action,Adventure,Comedy"
1,4972,movie,The Birth of a Nation,The Birth of a Nation,0,1915,None,195,"Drama,History,War"
2,10323,movie,The Cabinet of Dr. Caligari,Das Cabinet des Dr. Caligari,0,1920,None,76,"Fantasy,Horror,Mystery"


## 1. Finding all distinct film roles

In [8]:
query_q1 = """
SELECT DISTINCT category
FROM Role;
"""


res_q1 = pd.read_sql(query_q1, engine)
res_q1.head()

,category
0,actor
1,actress
2,writer
3,composer
4,editor


## 2. Computing number of film releases, annually

In [9]:
query_q2 = """
SELECT startYear AS year, COUNT(*) AS total
FROM Title
WHERE year IS NOT NULL
GROUP BY year
ORDER BY year DESC
LIMIT 10;
"""


res_q2 = pd.read_sql(query_q2, engine)
res_q2.head()

,year,total
0,2021,128
1,2020,181
2,2019,278
3,2018,282
4,2017,284


## 3. Reporting the number of "Big-Hit" and "Not-Big-Hit" films

In [10]:
query_q3 = """
SELECT
CASE
    WHEN CAST(numVotes AS int) > 100000 THEN 'yes'
    ELSE 'no'
END AS isBigHit,
COUNT(*) AS total
From Rating
INNER JOIN Title ON Rating.tconst = Title.tconst
GROUP BY isBigHit;
"""

res_q3 = pd.read_sql(query_q3, engine)
res_q3

,isBigHit,total
0,no,5341
1,yes,2233


## 4. Computing the average rating across films, per year

In [11]:
query_q4 = """
SELECT Title.startYear AS year, 
sum(averageRating * numVotes) / sum(numVotes) AS rating
FROM Rating
INNER JOIN Title ON Rating.tconst = Title.tconst
GROUP BY year
ORDER BY year DESC
LIMIT 10;
"""

res_q4 = pd.read_sql(query_q4, engine)
res_q4.head()

,year,rating
0,2021,7.036896
1,2020,7.004193
2,2019,7.333072
3,2018,7.083549
4,2017,7.196795


## 5. Finding people with the most ratings

In [12]:
query_q5 = """
CREATE VIEW PeopleRating AS
SELECT o.nconst, n.primaryName AS name, sum(r.numVotes) AS numVotes
From Rating AS r
INNER JOIN Role AS o ON r.tconst = o.tconst
INNER JOIN Name as n ON o.nconst = n.nconst
GROUP BY primaryName
ORDER BY numVotes DESC;
"""

#writing PeopleRating to db
engine.execute("DROP VIEW IF EXISTS PeopleRating")
engine.execute(query_q5)

res_q5 = pd.read_sql("SELECT * FROM PeopleRating LIMIT 10", engine)
res_q5

,nconst,name,numVotes
0,2369,John Williams,16574610
1,93,Brad Pitt,16007625
2,138,Leonardo DiCaprio,14757121
3,158,Tom Hanks,14703255
4,498278,Stan Lee,13906803
5,634240,Christopher Nolan,13728818
6,229,Steven Spielberg,13610887
7,134,Robert De Niro,13415268
8,168,Samuel L. Jackson,11997152
9,288,Christian Bale,11892101


## 6. Computing the number of A/B/C-list people per film

### 6a. Compute "X-List" classification per person

In [13]:
query_q6a = """
CREATE VIEW PeopleABC AS
SELECT numVotes > 1000000 AS isAList, 
numVotes < 999999 AND numVotes > 500000 AS isBList, 
numVotes < 499999 AND numVotes > 100000 AS isCList,
PeopleRating.nconst
FROM PeopleRating;
"""


#writing PeopleABC to db
engine.execute("DROP VIEW IF EXISTS PeopleABC")
engine.execute(query_q6a)

res_q6a = pd.read_sql("SELECT * FROM PeopleABC LIMIT 10", engine)
res_q6a

,isAList,isBList,isCList,nconst
0,1,0,0,2369
1,1,0,0,93
2,1,0,0,138
3,1,0,0,158
4,1,0,0,498278
5,1,0,0,634240
6,1,0,0,229
7,1,0,0,134
8,1,0,0,168
9,1,0,0,288


### 6b. Count up "X-List" people per film

In [14]:
query_q6b = """
SELECT sum(isAList) AS numAList, sum(isBList) AS numBList, sum(isCList) AS numCList, t.primaryTitle
FROM PeopleABC AS p
INNER JOIN Role AS r ON p.nconst = r.nconst
INNER JOIN Title as t ON r.tconst = t.tconst
GROUP BY t.tconst 
ORDER BY numAList DESC
LIMIT 10;
"""


res_q6b = pd.read_sql(query_q6b, engine)
res_q6b

,numAList,numBList,numCList,primaryTitle
0,10,0,0,The Godfather
1,10,0,0,The Godfather: Part II
2,10,0,0,Star Wars: Episode V - The Empire Strikes Back
3,10,0,0,Back to the Future
4,10,0,0,Die Hard
5,10,0,0,Goodfellas
6,10,0,0,The Silence of the Lambs
7,10,0,0,Terminator 2: Judgment Day
8,10,0,0,A Few Good Men
9,10,0,0,Forrest Gump


## 7. Finding pairs of people that have 50+ collaborations

### 7a. For each movie, listing all pairs of collaborators

In [15]:
query_q7a = """
CREATE VIEW Pair AS
SELECT r.nconst as nconst1,
o.nconst as nconst2,
r.tconst
FROM Role as r, Role as o
WHERE r.nconst > o.nconst AND r.tconst = o.tconst
"""


#writing Pair to db
engine.execute("DROP VIEW IF EXISTS Pair")
engine.execute(query_q7a)

res_q7a = pd.read_sql("SELECT * FROM Pair LIMIT 10", engine)
res_q7a.head()

,nconst1,nconst2,tconst
0,617588,29244,417
1,617588,76933,417
2,617588,242155,417
3,617588,324073,417
4,76933,29244,417


### 7b. Reporti number of movies per pair

In [16]:
query_q7b = """
SELECT a.primaryName as name1, b.primaryName as name2,
COUNT(*) AS total
FROM Pair AS p
INNER JOIN Name AS a ON a.nconst = p.nconst1
INNER JOIN Name AS b ON b.nconst = p.nconst2
GROUP BY name1, name2
HAVING total >= 50
ORDER BY total DESC
"""


res_q7b = pd.read_sql(query_q7b, engine)
res_q7b.head()

,name1,name2,total
0,D.B. Weiss,David Benioff,74
1,D.B. Weiss,George R.R. Martin,73
2,David Benioff,George R.R. Martin,73
3,D.B. Weiss,Peter Dinklage,59
4,David Benioff,Peter Dinklage,59
